<a id='report-interactive'></a>
<p style="font-weight:bold;"> <span style="font-size: 36px"> Interactive Reports</span> </p>

# Set up data and configuration

Choose to run the Reports notebook either with the set of Systemorph data in memory or with the data present in the Database: 
- #!eval-notebook "../Database/Configure" : connects to a physical Database
- #!eval-notebook "../Import/CloseImportTemplate" : uses the in-memory set up

We use here the in-memory set up.

In [0]:
#!import "../Import/CloseImportTemplate"

In [0]:
Workspace.InitializeFrom(DataSource);
//ifrs17Interactive.Reset(Workspace)

In [0]:
public class Ifrs17Interactive2 
{
    private Systemorph.Vertex.Pivot.Builder.Interfaces.IPivotFactory report;
    private Systemorph.InteractiveObjects.InteractiveObjectVariable interactiveObject;
    private ReportStorage storage;

    private IDictionary<string, Systemorph.Vertex.InteractiveObjects.InteractiveObjectView> interactiveObjectCache = new Dictionary<string, Systemorph.Vertex.InteractiveObjects.InteractiveObjectView>();

    public Ifrs17Interactive2 (IWorkspace workspace, Systemorph.Vertex.Pivot.Builder.Interfaces.IPivotFactory report, Systemorph.InteractiveObjects.InteractiveObjectVariable interactiveObject)
    {
        this.report = report;
        this.interactiveObject = interactiveObject;
        storage = new ReportStorage(workspace, report);
    }

    public void Reset(IWorkspace workspace) {
        storage = new ReportStorage(workspace, report);
        interactiveObjectCache = new Dictionary<string, Systemorph.Vertex.InteractiveObjects.InteractiveObjectView>();
    }

    public Systemorph.Vertex.InteractiveObjects.InteractiveObjectView GetFormsEntity<T>(string name = null) where T : ReportScope {
        var key = name ?? typeof(T).Name;
        if(!interactiveObjectCache.TryGetValue($"{key}FormsEntity", out var ret))
            ret = interactiveObjectCache[$"{key}FormsEntity"] = interactiveObject.CreateView($"{key}FormsEntity", _ => GetReportScope<T>(key));
        return ret;
    }

    public Systemorph.Vertex.InteractiveObjects.InteractiveObjectView GetReport<T>(string name = null) where T : ReportScope {
        var key = name ?? typeof(T).Name;
        if(!interactiveObjectCache.TryGetValue(key, out var ret))
            ret = interactiveObjectCache[key] = interactiveObject.CreateView(key,
                _ => {
                var scope = GetReportScope<T>(key);
                  var filters = scope.GetFilters(); // Not used and should be improved
                  return scope.ToReportAsync();
               });
        return ret;
    }

    public ReportScope GetReportScope<T>(string name = null) where T : ReportScope => interactiveObject.State.GetScope<T>(name ?? typeof(T).Name, o => o.WithStorage(storage));
}

In [0]:
var ifrs17Interactive2 = new Ifrs17Interactive2(Workspace, Report, InteractiveObject);
ifrs17Interactive2.Reset(Workspace)

In [0]:
ifrs17Interactive2.GetFormsEntity<PvReport>("A")

In [0]:
ifrs17Interactive2.GetReport<PvReport>("A")

In [0]:
ifrs17Interactive2.GetFormsEntity<PvReport>()

In [0]:
ifrs17Interactive2.GetReport<PvReport>()

In [0]:
InteractiveObject.CreateView("", _ => ifrs17Interactive2.GetReportScope<PvReport>())